Se importan las librerias necesarias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
sns.set()

Se lee el dataset

In [6]:
df_train = pd.read_parquet('train.parquet')
df_test = pd.read_parquet('test.parquet')

Teniendo en cuenta que las siguientes columnas no aportan nada al modelo, se las va a eliminar del dataset como primer filtro 
- 'id'
- 'url'
- 'region_url'
- 'image_url'
- 'description'

In [7]:
df_train.drop(['id', 'url', 'region_url', 'image_url', 'description'], axis=1, inplace=True)
df_test.drop(['id', 'url', 'region_url', 'image_url', 'description'], axis=1, inplace=True)

Por otro lado, considerando que las variables 'region', 'state', 'lat', 'long' estan intimamente relacionadas, se optara por eliminar estas ultimas 3

In [8]:
df_train.drop(['state', 'lat', 'long'], axis=1, inplace=True)
df_test.drop(['state', 'lat', 'long'], axis=1, inplace=True)

Se puede ver a continuacion que no hay duplicados

In [ ]:
#df_train[df_train.duplicated]

Se chequean los valores nulos. Se puede ver que es un dataset bastante completo sin cantidades considerables de valores nulos

In [ ]:
#msno.bar(df_train)
#plt.show()

Se analiza la correlacion entre columnas

In [ ]:
def plot_corre_heatmap(corr):
    '''
    Definimos una función para ayudarnos a graficar un heatmap de correlación
    '''
    plt.figure(figsize=(12,10))
    sns.heatmap(corr, cbar = True,  square = False, annot=True, fmt= '.2f'
                ,annot_kws={'size': 15},cmap= 'coolwarm')
    plt.xticks(rotation = 45)
    plt.yticks(rotation = 45)
    # Se hacen unos arreglos para una mejor visualizacion
    b, t = plt.ylim() # se define el valor inferior y superior
    b += 0.5          # Se agrega un 0.5 al valor inferior
    t -= 0.5          # Se le quita 0.5 al valor superior
    plt.ylim(b, t)    # Se actualiza el valor inferior y superior
    plt.show()

In [ ]:
#corr = df_train.corr()
#plot_corre_heatmap(corr)

A simple vista se puede ver que ninguna de las variables tiene una relacion considerable con la variable Preio

Por otro lado, se analizan los valores faltantes de las columnas 'parking_options' y 'laundry_options', los cuales parecen no tener razon alguna.

In [ ]:
#df_train['parking_options'].value_counts()

In [ ]:
#df_train['laundry_options'].value_counts()

Luego de haber visto las distribuciones de cada categoria en ambas variables, se ha determinado que se llenaran los valores nulos con la categoria predominante.
En el caso de 'parking_options' sera 'off-street parking' y para 'laundry_options' sera 'w/d in unit'

In [9]:
df_train['parking_options'].fillna('off-street parking', inplace=True)
df_train['laundry_options'].fillna('w/d in unit', inplace=True)

Se procesa la columna 'price' para convertirla en una variable categorica.
En primer lugar se eliminaran aquellos ingresos que se consideren outliers como ser los que tienen precio 0 y el que tenga un precio muy por encima del resto.
A continuacion se puede ver como los precios mas caros se encuentran en los millones y hay 1 solo de 27 trillones

In [ ]:
#df_train['price'].sort_values()

In [10]:
df_train = df_train[(df_train['price'] != 0) & (df_train['price'] !=2768307249)]

In [11]:
df_train = df_train.assign(category_price = pd.cut(df_train['price'], bins=[0,1000,2000,30000000], labels=['Low', 'Medium', 'High']))
df_train.drop(['price'], axis=1, inplace=True)

In [12]:
df_train['low_price'] = df_train['category_price'].map({'Low':1, 'Medium': 0,'High': 0})
df_train.drop(['category_price'], axis=1, inplace=True)

In [ ]:
df_train['low_price'].shape

(345303,)

Se procesan las variables nominales: region, type, laundry_options, parking_options.
Se generan variables dummies para cada una ellas

In [13]:
df_train_proc = pd.get_dummies(df_train, columns=['region', 'type', 'laundry_options', 'parking_options'])
df_test_proc = pd.get_dummies(df_train, columns=['region', 'type', 'laundry_options', 'parking_options'])

Por ultimo, se hace un reseteo de los indexes

In [14]:
df_train_proc.reset_index(drop=True, inplace=True)

    Una vez que el dataset esta preprocesado, se pasa al modelado.
    En este proceso se ha elegido trabajar con el metodo Kneighbors dada la estructura del dataset

In [15]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

Instanciamos el modelo

In [16]:
modelo = KNeighborsClassifier(n_neighbors= 3)

X = df_train_proc.drop(['low_price'],axis=1)
y = df_train_proc['low_price']

In [17]:
y.shape, X.shape

((345303,), (345303, 437))

Se va a entrenar el modelo con validacion cruzada con un valor de folio de 5. Es decir, en cada validacion habra un 20% test y un 80% train.

In [25]:
X.iloc[[921]]

,sqfeet,beds,baths,cats_allowed,dogs_allowed,smoking_allowed,wheelchair_access,electric_vehicle_charge,comes_furnished,region_SF bay area,...,laundry_options_no laundry on site,laundry_options_w/d hookups,laundry_options_w/d in unit,parking_options_attached garage,parking_options_carport,parking_options_detached garage,parking_options_no parking,parking_options_off-street parking,parking_options_street parking,parking_options_valet parking
921,600,2,1.0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [18]:
modelo_vc = cross_val_score(modelo, X, y, cv=5, error_score='raise')

MemoryError: Unable to allocate 921. MiB for an array with shape (276242, 437) and data type float64